In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/wordcount
!hdfs dfs -ls /tmp/

Found 3 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 19:24 /tmp/output
Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 19:24 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 19:30 /tmp/wordcount


In [4]:
!apt-get update && apt-get install wget

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Get:2 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]               
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Get:5 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [30.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic/main amd64 Packages [1344 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [7064 B]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [822 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [834 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic/restricted amd64 Packages [13.5 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic/universe amd64 Packages [11.3 MB]
Get:12 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 Packages [186 kB]
G

In [3]:
!hdfs dfs -copyFromLocal *.csv  /tmp/wordcount/

In [4]:
!hdfs dfs -ls /tmp/wordcount/

Found 2 items
-rw-r--r--   1 root supergroup        397 2020-02-18 19:38 /tmp/wordcount/tbl0.csv
-rw-r--r--   1 root supergroup        282 2020-02-18 19:38 /tmp/wordcount/tbl1.csv


In [5]:
!hdfs dfs -cat /tmp/wordcount/tbl0.csv

1,D,4,2016-06-25,a:e:c:d,bb#10:dd#20:cc#40
2,C,4,2015-05-14,a:c,dd#40:bb#20:cc#10
3,D,6,2014-12-26,b:d,aa#10:bb#40
4,D,5,2016-06-25,a:c:e:b:d,bb#40:dd#20:aa#10:cc#30
5,C,7,2016-05-23,d:e:c,cc#10:aa#20
6,A,2,2018-06-14,a:d,aa#20
7,B,3,2014-12-22,a:e:d,cc#20
8,C,6,2015-08-20,d:a:c:e,cc#10:aa#20
9,B,3,2017-12-08,b:a:c:e,cc#40:dd#10:aa#30:bb#20
10,B,7,2015-07-28,c:d:e:a:b,aa#10:dd#40:cc#30

In [6]:
%%hive

DROP TABLE IF EXISTS tbl0;
DROP TABLE IF EXISTS tbl1;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 13.767 seconds
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.013 seconds


In [7]:
%%hive

CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.51 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 2.512 seconds


In [8]:
%%hive

CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.108 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 1.176 seconds


In [16]:
%%hive
DROP TABLE IF EXISTS unique_values;
CREATE TABLE unique_values AS
SELECT DISTINCT a.*
FROM (SELECT explode(c5) FROM tbl0) a;

DROP TABLE IF EXISTS unique_values;
OK
Time taken: 0.014 seconds
CREATE TABLE unique_values AS
SELECT DISTINCT a.*
FROM (SELECT explode(c5) FROM tbl0) a;
Query ID = root_20200218194847_e172192d-d74a-49c5-b086-b8601374be0a
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582053079980_0008, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0008/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0008
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-18 19:48:58,031 Stage-1 map = 0%,  reduce = 0%
2020-02-18 19:49:03,781 Stage-1 map = 100%,  reduc

In [17]:
%%hive
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM unique_values;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM unique_values;
Query ID = root_20200218194928_e6eaa6b3-6f0a-4d8c-821d-f789e8d11be0
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582053079980_0009, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0009/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0009
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-18 19:49:38,640 Stage-1 map = 0%,  reduce = 0%
2020-02-18 19:49:44,470 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 1.87 sec
MapReduce Total cumulative CPU time: 1 seconds 870 msec
Ended Job = job_1582053079980_0009
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/tmp/output/.hive-stagin

In [18]:
!hive -S -e 'source output.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
File: output.hql is not a file.


In [19]:
!hdfs dfs -cat /tmp/output/000000_0 | head

a
b
c
d
e


In [20]:
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [21]:
!cat output/000000_0

a
b
c
d
e
